<a href="https://colab.research.google.com/github/khal-drog0/Breast_Cancer_Histopathology_Classification/blob/main/Drive_After_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip3 install -q kaggle

from google.colab import files
files.upload()

! mkdir ~/.kaggle

! cp kaggle.json /root/.kaggle

! chmod 600 /root/.kaggle/kaggle.json

! kaggle datasets download -d paultimothymooney/breast-histopathology-images

! unzip breast-histopathology-images.zip

Streaming output truncated to the last 5000 lines.
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1551_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1601_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1651_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1701_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1751_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1801_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1851_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1901_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1951_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2001_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2251_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2301_class0.png  
  inflating: IDC_regular_

In [ ]:
import numpy as np
import pandas as pd
import os
import itertools
import os, stat, time
from os.path import dirname as up
import shutil
import glob
from PIL import Image

In [ ]:
from glob import glob
data = glob('IDC_regular_ps50_idx5/**/*.png', recursive = True)

In [ ]:
img_size = 32
num_channels = 3

In [ ]:
import random

def randomSamples(imageData):
    lst = list()
    while(len(lst) != 15000):
        a = random.choice(imageData)

        if a not in lst:
            lst.append(a)

    return lst

In [ ]:
import cv2
import matplotlib.pyplot as plt

images = list()
labels = list()
newData = list()

newData = randomSamples(data)

for i in newData:
    if i.endswith('.png'):
        label = i[-5]
        img = cv2.imread(i)
        img_1 = cv2.resize(img, (img_size, img_size))
        images.append(img_1)
        labels.append(label)

In [ ]:
x = np.stack(images)

In [ ]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(labels)

In [ ]:
x = x / 255

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 0, test_size = 0.2)

# Building VAE

Building the encoder

In [ ]:
import tensorflow

In [ ]:
a = tensorflow.keras.layers.Input(shape = (img_size, img_size, num_channels), name = 'encoder_input')

encoder_conv_layer1 = tensorflow.keras.layers.Conv2D(filters = 1, kernel_size = (3, 3), padding = 'same', strides = 1, name = 'encoder_conv_1')(a)
encoder_norm_layer1 = tensorflow.keras.layers.BatchNormalization(name = 'encoder_norm_1')(encoder_conv_layer1)
encoder_activ_layer1 = tensorflow.keras.layers.LeakyReLU(name = 'encoder_leakyrelu_1')(encoder_norm_layer1)

encoder_conv_layer2 = tensorflow.keras.layers.Conv2D(filters = 32, kernel_size = (3, 3), padding = 'same', strides = 1, name = 'encoder_conv_2')(encoder_activ_layer1)
encoder_norm_layer2 = tensorflow.keras.layers.BatchNormalization(name="encoder_norm_2")(encoder_conv_layer2)
encoder_activ_layer2 = tensorflow.keras.layers.LeakyReLU(name="encoder_activ_layer_2")(encoder_norm_layer2)

encoder_conv_layer3 = tensorflow.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", strides=2, name="encoder_conv_3")(encoder_activ_layer2)
encoder_norm_layer3 = tensorflow.keras.layers.BatchNormalization(name="encoder_norm_3")(encoder_conv_layer3)
encoder_activ_layer3 = tensorflow.keras.layers.LeakyReLU(name="encoder_activ_layer_3")(encoder_norm_layer3)

encoder_conv_layer4 = tensorflow.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", strides=2, name="encoder_conv_4")(encoder_activ_layer3)
encoder_norm_layer4 = tensorflow.keras.layers.BatchNormalization(name="encoder_norm_4")(encoder_conv_layer4)
encoder_activ_layer4 = tensorflow.keras.layers.LeakyReLU(name="encoder_activ_layer_4")(encoder_norm_layer4)

encoder_conv_layer5 = tensorflow.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", strides=1, name="encoder_conv_5")(encoder_activ_layer4)
encoder_norm_layer5 = tensorflow.keras.layers.BatchNormalization(name="encoder_norm_5")(encoder_conv_layer5)
encoder_activ_layer5 = tensorflow.keras.layers.LeakyReLU(name="encoder_activ_layer_5")(encoder_norm_layer5)

shape_before_flatten = tensorflow.keras.backend.int_shape(encoder_activ_layer5)[1:]
encoder_flatten = tensorflow.keras.layers.Flatten()(encoder_activ_layer5)

In [ ]:
latent_space_dim = 2

encoder_mu = tensorflow.keras.layers.Dense(units = latent_space_dim, name = 'encoder_mu')(encoder_flatten)
encoder_log_variance = tensorflow.keras.layers.Dense(units = latent_space_dim, name = 'encoder_log_variance')(encoder_flatten)

In [ ]:
def sampling(mu_log_variance):
    mu, log_variance = mu_log_variance
    epsilon = tensorflow.keras.backend.random_normal(shape = tensorflow.keras.backend.shape(mu), mean = 0.0, stddev = 1.0)
    random_sample = mu + tensorflow.keras.backend.exp(log_variance / 2) * epsilon

    return random_sample

In [ ]:
encoder_output = tensorflow.keras.layers.Lambda(sampling, name = 'encoder_output')([encoder_mu, encoder_log_variance])

In [ ]:
encoder = tensorflow.keras.models.Model(a, encoder_output, name = 'encoder_model')

In [ ]:
encoder.summary()

Model: "encoder_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 encoder_conv_1 (Conv2D)        (None, 32, 32, 1)    28          ['encoder_input[0][0]']          
                                                                                                  
 encoder_norm_1 (BatchNormaliza  (None, 32, 32, 1)   4           ['encoder_conv_1[0][0]']         
 tion)                                                                                            
                                                                                                  
 encoder_leakyrelu_1 (LeakyReLU  (None, 32, 32, 1)   0           ['encoder_norm_1[0][0

Building the decoder

In [ ]:
decoder_input = tensorflow.keras.layers.Input(shape = (latent_space_dim), name = 'decoder_input')

decoder_dense_layer1 = tensorflow.keras.layers.Dense(units = np.prod(shape_before_flatten), name = 'decoder_dense_1')(decoder_input)

decoder_reshape = tensorflow.keras.layers.Reshape(target_shape = shape_before_flatten)(decoder_dense_layer1)

decoder_conv_tran_layer1 = tensorflow.keras.layers.Conv2DTranspose(filters=64, kernel_size=(3, 3), padding="same", strides=1, name="decoder_conv_tran_1")(decoder_reshape)
decoder_norm_layer1 = tensorflow.keras.layers.BatchNormalization(name="decoder_norm_1")(decoder_conv_tran_layer1)
decoder_activ_layer1 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_1")(decoder_norm_layer1)

decoder_conv_tran_layer2 = tensorflow.keras.layers.Conv2DTranspose(filters=64, kernel_size=(3, 3), padding="same", strides=2, name="decoder_conv_tran_2")(decoder_activ_layer1)
decoder_norm_layer2 = tensorflow.keras.layers.BatchNormalization(name="decoder_norm_2")(decoder_conv_tran_layer2)
decoder_activ_layer2 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_2")(decoder_norm_layer2)

decoder_conv_tran_layer3 = tensorflow.keras.layers.Conv2DTranspose(filters=64, kernel_size=(3, 3), padding="same", strides=2, name="decoder_conv_tran_3")(decoder_activ_layer2)
decoder_norm_layer3 = tensorflow.keras.layers.BatchNormalization(name="decoder_norm_3")(decoder_conv_tran_layer3)
decoder_activ_layer3 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_3")(decoder_norm_layer3)

decoder_conv_tran_layer4 = tensorflow.keras.layers.Conv2DTranspose(filters=1, kernel_size=(3, 3), padding="same", strides=1, name="decoder_conv_tran_4")(decoder_activ_layer3)
decoder_output = tensorflow.keras.layers.LeakyReLU(name="decoder_output")(decoder_conv_tran_layer4 )

In [ ]:
decoder = tensorflow.keras.models.Model(decoder_input, decoder_output, name = 'decoder_model')

In [ ]:
decoder.summary()

Model: "decoder_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 2)]               0         
                                                                 
 decoder_dense_1 (Dense)     (None, 4096)              12288     
                                                                 
 reshape_1 (Reshape)         (None, 8, 8, 64)          0         
                                                                 
 decoder_conv_tran_1 (Conv2D  (None, 8, 8, 64)         36928     
 Transpose)                                                      
                                                                 
 decoder_norm_1 (BatchNormal  (None, 8, 8, 64)         256       
 ization)                                                        
                                                                 
 decoder_leakyrelu_1 (LeakyR  (None, 8, 8, 64)       

Building the VAE

In [ ]:
vae_input = tensorflow.keras.layers.Input(shape = (img_size, img_size, num_channels), name = 'VAE_input')

vae_encoder_output = encoder(vae_input)

vae_decoder_output = decoder(vae_encoder_output)

vae = tensorflow.keras.models.Model(vae_input, vae_decoder_output, name = 'VAE')

In [ ]:
vae.summary()

Model: "VAE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 VAE_input (InputLayer)      [(None, 32, 32, 3)]       0         
                                                                 
 encoder_model (Functional)  (None, 2)                 109988    
                                                                 
 decoder_model (Functional)  (None, 32, 32, 1)         124417    
                                                                 
Total params: 234,405
Trainable params: 233,571
Non-trainable params: 834
_________________________________________________________________


In [ ]:
def loss_func(encoder_mu, encoder_log_variance):

    def vae_reconstruction_loss(y_true, y_predict):
        reconstruction_loss_factor = 1000
        reconstruction_loss = tensorflow.keras.backend.mean(tensorflow.keras.backend.square(y_true - y_predict), axis = [1, 2, 3])

        return reconstruction_loss_factor * reconstruction_loss
    
    def vae_kl_loss(encoder_mu, encoder_log_variance):
        kl_loss = -0.5 * tensorflow.keras.backend.sum(1.0 + encoder_log_variance - tensorflow.keras.backend.square(encoder_mu) - tensorflow.keras.backend.exp(encoder_log_variance), axis = [1, 2, 3])
        return kl_loss

    def vae_kl_loss_metric(y_true, y_predict):
        kl_loss = -0.5 * tensorflow.keras.backend.sum(1.0 + encoder_log_variance - tensorflow.keras.backend.square(encoder_mu) - tensorflow.keras.backend.exp(encoder_log_variance), axis = [1, 2, 3])
        return kl_loss

    def vae_loss(y_true, y_predict):
        reconstruction_loss = vae_reconstruction_loss(y_true, y_predict)
        kl_loss = vae_kl_loss(y_true, y_predict)

        loss = reconstruction_loss + kl_loss
        return loss

    return vae_loss

In [ ]:
vae.compile(optimizer = tensorflow.keras.optimizers.Adam(lr = 0.00001), loss = loss_func(encoder_mu, encoder_log_variance))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
x_train = np.reshape(x_train, newshape=(x_train.shape[0], x_train.shape[1], x_train.shape[2], 3)) 
x_test = np.reshape(x_test, newshape=(x_test.shape[0], x_train.shape[1], x_train.shape[2], 3))

In [ ]:
history = vae.fit(x_train, x_train, epochs = 25, batch_size = 64, shuffle = True, validation_data = (x_test, x_test))

Epoch 1/25
188/188 [==============================] - 7s 36ms/step - loss: 1134.2584 - val_loss: 1135.1808 - lr: 1.0000e-05
Epoch 2/25
188/188 [==============================] - 7s 35ms/step - loss: 1134.2642 - val_loss: 1135.1619 - lr: 1.0000e-05
Epoch 3/25
188/188 [==============================] - 6s 34ms/step - loss: 1134.2166 - val_loss: 1135.1195 - lr: 1.0000e-05
Epoch 4/25
188/188 [==============================] - 6s 34ms/step - loss: 1134.2145 - val_loss: 1135.1606 - lr: 1.0000e-05
Epoch 5/25
187/188 [============================>.] - ETA: 0s - loss: 1134.3887
Epoch 5: ReduceLROnPlateau reducing learning rate to 1.9999999494757505e-06.
188/188 [==============================] - 6s 34ms/step - loss: 1134.1798 - val_loss: 1135.1198 - lr: 1.0000e-05
Epoch 6/25
188/188 [==============================] - 6s 34ms/step - loss: 1134.1659 - val_loss: 1135.1169 - lr: 2.0000e-06
Epoch 7/25
188/188 [==============================] - 6s 35ms/step - loss: 1134.1559 - val_loss: 1135.0963 - l

In [ ]:
encoder.save("VAE_encoder.h5") 
decoder.save("VAE_decoder.h5") 
vae.save("VAE.h5")

In [ ]:
# Testing the encoder and decoder

encoded_data = encoder.predict(x_test)
decoded_data = decoder.predict(encoded_data)

In [ ]:
decoded_data[1]